<a href="https://colab.research.google.com/github/adriansuder/NLP_WSEI/blob/main/NLP_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zadanie 1 BagOfWords

In [1]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups

# Download the stopwords for the 'polish' language

# Wybieramy kilka kategorii do analizy
categories = ['sci.med', 'sci.space', 'rec.sport.baseball', 'comp.graphics']
newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data

print(f"Pobrano {len(documents)} dokumentów.")

vectorizer = CountVectorizer(stop_words='english')
bow_matrix = vectorizer.fit_transform(documents)
vocabulary = vectorizer.get_feature_names_out()

X = bow_matrix.toarray()

# Wyświetlenie macierzy Bag-of-Words
print("\nMacierz Bag-of-Words:")
print(X)

# Sumowanie wystąpień słów w kolumnach
word_counts = np.sum(X, axis=0)

# Tworzenie listy par (słowo, liczebność)
word_count_pairs = list(zip(vocabulary, word_counts))

# Sortowanie listy według liczebności (malejąco)
word_count_pairs.sort(key=lambda x: x[1], reverse=True)

print("15 Najczęściej występujących słów:")
for word, count in word_count_pairs[:15]:
    print(f"{word}: {count}")

Pobrano 2368 dokumentów.

Macierz Bag-of-Words:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
15 Najczęściej występujących słów:
space: 1060
edu: 835
like: 743
don: 722
just: 642
time: 623
use: 606
know: 594
think: 560
good: 548
year: 546
image: 530
new: 488
data: 487
people: 476


# Zadanie 2 tf-idf

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups

# Wybieramy kilka kategorii do analizy
categories = ['sci.med', 'sci.space', 'rec.sport.baseball', 'comp.graphics']
newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data

print(f"Pobrano {len(documents)} dokumentów.")

# Inicjalizacja TfidfVectorizer z polskimi stop-słowami
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Pobranie słownika
vocabulary = vectorizer.get_feature_names_out()

# Wyświetlenie macierzy TF-IDF dla wszystkich dokumentów
print("\nMacierz TF-IDF:")
print(tfidf_matrix.toarray())

#index dla którego wyświetlimy tf-idf
doc_index = 2

# Pobranie wartości TF-IDF dla wybranego dokumentu
tfidf_values = tfidf_matrix[doc_index].toarray()[0]

# Tworzenie DataFrame z słowami i ich wartościami TF-IDF
tfidf_df = pd.DataFrame({'word': vocabulary, 'tfidf': tfidf_values})

# Sortowanie DataFrame według wartości TF-IDF (malejąco)
tfidf_df_sorted = tfidf_df.sort_values(by='tfidf', ascending=False)

print("Wybrany dokument dla którego wyświetli się tf-idf")
print(documents[doc_index])

# Wyświetlanie 10 słów o najwyższych wartościach TF-IDF
print(f"\nSłowa o najwyższych wartościach TF-IDF dla dokumentu {doc_index}:")
print(tfidf_df_sorted.head(10))



Pobrano 2368 dokumentów.

Macierz TF-IDF:
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.13300203 0.         0.         ... 0.         0.         0.        ]]
Wybrany dokument dla którego wyświetli się tf-idf


Sjogren's syndrome has been known to induce dryness in vaginal tissue as well
as induce primary biliary cirrhosis. Otherwise the abdominal swelling could be
due to a complication of Sjogren's known as pseudolymphoma which *can* produce
a splenomegaly (enlarged spleen). She should definitely see a rheumatologist.

Since you don't mention skin disorder, anemia, or joint pain you'd probably
rule out erythema nodosum or scleroderma.

Josh
backon@VMS.HUJI.AC.IL



# Zadanie 3 word embdeddings

In [3]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.datasets import fetch_20newsgroups
from gensim.models import Word2Vec
import string


# Wybieramy kilka kategorii do analizy
categories = ['sci.med', 'sci.space', 'rec.sport.baseball', 'comp.graphics']
newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data

print(f"Pobrano {len(documents)} dokumentów.")

# Tokenizacja, usuwanie stop-słów i normalizacja
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and word.isalpha()]
    return tokens

processed_documents = [preprocess(doc) for doc in documents]

# Trenowanie modelu Word2Vec
model = Word2Vec(sentences=processed_documents, vector_size=100, window=5, min_count=5, workers=4)

# Testowanie modelu, do word podajemy słowo dla którego znajdziemy podobne słowa
word = "space"
similar_words = model.wv.most_similar(word, topn=5)

print(f"Najbliższe słowa do '{word}':")
for similar_word, similarity in similar_words:
    print(f"{similar_word}: {similarity:.4f}")



Pobrano 2368 dokumentów.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Najbliższe słowa do 'space':
center: 0.9885
nasa: 0.9842
national: 0.9819
research: 0.9781
data: 0.9760


# Zadanie 4 bigramy

In [5]:
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter

# Wybieramy kilka kategorii do analizy
categories = ['sci.med', 'sci.space', 'rec.sport.baseball', 'comp.graphics']
newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data

print(f"Pobrano {len(documents)} dokumentów.")

nltk.download('punkt')
nltk.download('stopwords')

def preprocess(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and word.isalpha()]
    return tokens

processed_documents = [preprocess(doc) for doc in documents]

# Generowanie bigramów
bigramy = []
for dokument in processed_documents:
  for i in range(len(dokument) - 1):
    bigramy.append((dokument[i], dokument[i + 1]))

licznik_bigramow = Counter(bigramy)

# Wyświetlanie n najczęściej występujących bigramów
print(f"Najczęściej występujące bigramy (top 25):")
for bigram, liczba_wystapien in licznik_bigramow.most_common(25):
    print(f"{bigram}: {liczba_wystapien}")





Pobrano 2368 dokumentów.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Najczęściej występujące bigramy (top 25):
('dont', 'know'): 124
('last', 'year'): 110
('new', 'york'): 101
('im', 'sure'): 87
('gordon', 'banks'): 80
('anonymous', 'ftp'): 80
('banks', 'skepticism'): 73
('skepticism', 'chastity'): 73
('chastity', 'intellect'): 73
('intellect', 'gebcadredslpittedu'): 72
('gebcadredslpittedu', 'shameful'): 72
('shameful', 'surrender'): 72
('surrender', 'soon'): 72
('dont', 'think'): 71
('would', 'like'): 69
('space', 'station'): 68
('years', 'ago'): 59
('image', 'processing'): 56
('anyone', 'know'): 55
('medical', 'newsletter'): 55
('space', 'shuttle'): 54
('hicnet', 'medical'): 54
('newsletter', 'page'): 54
('volume', 'number'): 54
('number', 'april'): 54


# Zadanie 5 Analiza trigramów

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams
import string
from collections import Counter

# Wybieramy kilka kategorii do analizy
categories = ['sci.med', 'sci.space', 'rec.sport.baseball', 'comp.graphics']
newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data

print(f"Pobrano {len(documents)} dokumentów.")

nltk.download('punkt')
nltk.download('stopwords')

def preprocess(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and word.isalpha()]
    return tokens

processed_documents = [preprocess(doc) for doc in documents]

# Generowanie trigramów z użycie ntlk
trigramy = []
for dokument in processed_documents:
  ngrams_temp = ngrams(dokument, 3)
  for ngram in ngrams_temp:
    trigramy.append(ngram)

licznik_triramow = Counter(trigramy)

# Wyświetlanie n najczęściej występujących trigramów
print(f"Najczęściej występujące trigramy (top 25):")
for trigram, liczba_wystapien in licznik_triramow.most_common(25):
    print(f"{trigram}: {liczba_wystapien}")